In [2]:
import numpy as np
from numpy.typing import NDArray

In [40]:
n, N, r = 7, 10, 4
num_iterations = 100

In [4]:
def e(k, r):
    if k > r:
        raise ValueError("k should be less than or equal to r")
    
    # Create a zero vector of size r
    e_kr = np.zeros(r)
    
    # Set the k-th entry to 1
    e_kr[k] = 1
    
    return e_kr

In [ ]:
def h_k(gamma, N, y, t, lambdas, W, ):
    return (1/(2*N))*np.linalg.norm(y, ord=2)

In [5]:
# Inputs
X = np.random.random((n, N)) # shape (n, N)
y = np.zeros(N) # shape (N)

In [32]:
# Inicialization

U = np.concatenate([e(i, n).reshape(-1, 1) for i in range(r)], axis=-1)

P = np.eye(r) # shape (r,r)
UQ = np.concatenate([e(i, n).reshape(-1, 1) for i in range(r)], axis=-1)
lambdas = np.ones(r) # shape (r)
W = X.T @ np.concatenate([e(i, n).reshape(-1, 1) for i in range(r)], axis=-1) # shape (N, r)
t = [X[:, p].T @ U @ P @ U.T @ X[:, p] for p in range(N)]

In [33]:
assert (np.all([np.isclose(t[p], np.sum(X[:, p][:r]**2)) for p in range(N)]))

In [34]:
def v_hat(k):
    return (2*lambdas[k]/N)*(X @ ((y-t)*W[:, k]) - UQ @ (W.T @ ((y-t)*W[:, k])))

In [35]:
def v(k):
    v_hat = v_hat(k)
    
    return v_hat/np.linalg.norm(v_hat)**2

In [47]:
def U_plus(gamma, v, k):
    return U + ((np.cos(gamma)-1)*UQ[:, k] + np.sin(gamma)*v)*UQ[:, k].T

In [43]:
def h(gamma, k):
    
    v_hat = (2*lambdas[k]/N)*(X @ ((y-t)*W[:, k]) - UQ @ (W.T @ ((y-t)*W[:, k])))
            
    v = v_hat/np.linalg.norm(v_hat)**2
    z = X.T @ v
    s = (np.cos(gamma)-1)*W[:, k] + np.sin(gamma)*z
    return (1/(2*N)) * np.linalg.norm(y-t-2*lambdas[k]*s - lambdas[k]*(s*s), ord=2)**2

In [44]:
def find_optimal_gamma(k):
    num_points = 1000
    x_values = np.linspace(0, 2*np.pi, num_points)
    y_values = [h(x, k) for x in x_values]

    # Find the x value that minimizes the objective function
    optimal_x = x_values[np.argmin(y_values)]
    optimal_y = np.min(y_values)
    
    return optimal_x

In [48]:
for t in range(num_iterations):
    for l in range(n):
        for k in range(r):
            v_hat = (2*lambdas[k]/N)*(X @ ((y-t)*W[:, k]) - U @ Q @ (W.T @ ((y-t)*W[:, k])))
            
            v = v_hat/np.linalg.norm(v_hat)**2
            
            z = X.T @ v
            
            optimal_gamma = find_optimal_gamma(k)
            
            U = U_plus(optimal_gamma, v, k)
            P = P
            
            s = (np.cos(optimal_gamma)-1)*W[:, k] + np.sin(optimal_gamma)*z
            
            W[:, k] = X.T @ (s + UQ[:, k])
            t = t + 2*lambdas[k]*s + lambdas[k]*(s*s)
            UQ[:, k] += s

/var/folders/7x/w344fx1j18q71m51_grbtgjc0000gn/T/ipykernel_75373/1188394085.py:6: RuntimeWarning: invalid value encountered in divide
  v = v_hat/np.linalg.norm(v_hat)**2
/var/folders/7x/w344fx1j18q71m51_grbtgjc0000gn/T/ipykernel_75373/1574969654.py:5: RuntimeWarning: invalid value encountered in divide
  v = v_hat/np.linalg.norm(v_hat)**2


ValueError: operands could not be broadcast together with shapes (7,4) (7,) 